# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [1]:
import numpy as np
import pandas as pd

import platform
if platform.system() == "Windows":
    BASE_DIR = r'C:\Users\Alex\Desktop\Academic\UChicago\FINM 37500\finm-fiderivs-2024'
else:
    BASE_DIR = r'/Users/alexhuang/Documents/Academic/FINM 37500/finm-fiderivs-2024'


import sys, os
sys.path.append(BASE_DIR)

from cmds.binomial import *
from cmds.ratecurves import *
from cmds.volskew import *
from cmds.ficcvol import *
from cmds.fi_binomial_model import FIBinomialModel

In [2]:
vol = pd.read_excel('../data/swaption_vol_data.xlsx')
vol

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


In [3]:
cap_curves = pd.read_excel('../data/cap_curves_2024-02-20.xlsx')
cap_curves.set_index('tenor', inplace=True)
cap_curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247
1.50,0.046235,0.046141,0.933499,0.039030,0.292615,0.380930
1.75,0.045059,0.044939,0.924774,0.037738,0.315878,0.388953
2.00,0.044133,0.043994,0.916212,0.037382,0.331443,0.386643
2.25,0.043173,0.043011,0.908230,0.035151,0.340919,0.376247


## 1.1

In [4]:
start = 1
end = start + 4

forward_swap_rate = calc_fwdswaprate(cap_curves['discounts'], start, end, freqswap=4)
forward_swap_rate

0.03672212061985555

## 1.2

In [5]:
strikerange = np.array(vol.columns[-9:].tolist())
vols = vol[strikerange] / 100
strikes = forward_swap_rate + strikerange/100/100
# display(strikerange)
# display(vols)
# display(forward_swap_rate)
# display(strikes)

period_fwd = cap_curves.index.get_loc(start)
period_swap = cap_curves.index.get_loc(end) + 1
# display(period_fwd)
# display(period_swap)

discount = cap_curves['discounts'].iloc[period_fwd+1 : period_swap].sum() / 4
print(cap_curves['discounts'].iloc[period_fwd+1 : period_swap])
# display(cap_curves.index[period_fwd+1])
# display(cap_curves.index[period_swap])
# display(discount)
# print(start, vols, strikes, forward_swap_rate, discount, sep='\n')

quotes = vols.copy()
quotes.loc['price'] = 100 * blacks_formula(start, vols, strikes, forward_swap_rate, discount, isCall=True)[0]
quotes.loc['strike'] = strikes
quotes = quotes.rename(index={0: 'implied vol'})
quotes = quotes.loc[['strike', 'implied vol', 'price']]
quotes

tenor
1.25    0.942608
1.50    0.933499
1.75    0.924774
2.00    0.916212
2.25    0.908230
2.50    0.900188
2.75    0.892076
3.00    0.883906
3.25    0.876134
3.50    0.868539
3.75    0.860726
4.00    0.852910
4.25    0.845410
4.50    0.838026
4.75    0.830464
5.00    0.822816
Name: discounts, dtype: float64


,-200,-100,-50,-25,0,25,50,100,200
strike,0.016722,0.026722,0.031722,0.034222,0.036722,0.039222,0.041722,0.046722,0.056722
implied vol,0.545400,0.403700,0.359400,0.342300,0.328300,0.317100,0.308600,0.298300,0.295400
price,7.203088,4.065315,2.739848,2.174850,1.687383,1.281199,0.955570,0.513281,0.147265


## 1.3

In [6]:
expiries = [.25,2,1]
tenors = [4,4,2]
fwdswaps = np.full(len(expiries), np.nan)

quotes_alt = pd.DataFrame(columns=['expiry', 'tenor', 'price'])

idstrikeATM = np.where(strikerange==0)[0][0]
# display(idstrikeATM)
strikeATM = strikes[idstrikeATM]
# display(strikeATM)
volATM = vols.iloc[0,idstrikeATM]
# display(volATM)

for i in range(len(fwdswaps)):
    print(expiries[i], expiries[i]+tenors[i], sep='\n')
    fwdswaps[i] = calc_fwdswaprate(cap_curves['discounts'], expiries[i], expiries[i]+tenors[i], freqswap=4)
    
    period0 = cap_curves.index.get_loc(expiries[i])
    period1 = cap_curves.index.get_loc(expiries[i]+tenors[i])+1
    
    # print(period0, period1)
    # print(expiries[i])
    discount = cap_curves['discounts'].iloc[period0+1 : period1].sum() / 4
    # print(cap_curves['discounts'].iloc[period0+1 : period1])
    # print(discount)
    # print(expiries[i], volATM, strikeATM, forward_swap_rate, discount, sep='\n')
    # print()
    quotes_alt.loc[i,['expiry', 'tenor']] = [expiries[i], tenors[i]]
    quotes_alt.loc[i, 'price'] = 100 * blacks_formula(expiries[i], volATM, strikeATM, forward_swap_rate, discount, isCall=True)
    
quotes_alt

0.25
4.25
2
6
1
3


,expiry,tenor,price
0,0.25,4,0.870475
1,2,4,2.291364
2,1,2,0.874004


***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [7]:
sig = cap_curves['fwd vols']
sig.iloc[0] = sig.iloc[1]
sig

tenor
0.25     0.166025
0.50     0.166025
0.75     0.210648
1.00     0.254312
1.25     0.361247
1.50     0.380930
1.75     0.388953
2.00     0.386643
2.25     0.376247
2.50     0.363764
2.75     0.353513
3.00     0.349207
3.25     0.350339
3.50     0.351556
3.75     0.349233
4.00     0.339501
4.25     0.322525
4.50     0.309744
4.75     0.306238
5.00     0.311414
5.25     0.324078
5.50     0.332488
5.75     0.331264
6.00     0.317965
6.25     0.294203
6.50     0.273998
6.75     0.262325
7.00     0.260895
7.25     0.268693
7.50     0.275091
7.75     0.276397
8.00     0.271592
8.25     0.261350
8.50     0.251699
8.75     0.244536
9.00     0.240251
9.25     0.239254
9.50     0.241978
9.75     0.248893
10.00    0.260524
Name: fwd vols, dtype: float64

In [8]:
disc = cap_curves['discounts']
theta_params, rate_tree = estimate_theta(sig, disc)

In [9]:
pd.DataFrame(theta_params).T

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
theta,NaN,-0.117846,-0.262526,-0.359865,-0.720504,-0.459257,-0.384915,-0.244081,-0.396789,-0.033089,...,0.125541,0.134603,0.153908,0.169295,0.178789,0.182346,0.179249,0.168698,0.149782,0.121431


In [10]:
# Continuously
rate_tree = rate_tree[[col for col in rate_tree.columns if col <= 5]]
rate_tree = rate_tree.dropna(how='all')
rate_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,0.052,0.055,0.057,0.059,0.059,0.064,0.070,0.080,0.088,0.104,...,0.150,0.171,0.200,0.247,0.298,0.343,0.406,0.500,0.608,0.721
1,NaN,0.046,0.048,0.050,0.050,0.054,0.060,0.068,0.074,0.088,...,0.127,0.145,0.170,0.210,0.252,0.291,0.344,0.424,0.515,0.611
2,NaN,NaN,0.039,0.041,0.041,0.044,0.048,0.055,0.060,0.072,...,0.103,0.117,0.137,0.170,0.204,0.236,0.279,0.343,0.417,0.495
3,NaN,NaN,NaN,0.031,0.031,0.034,0.037,0.043,0.047,0.056,...,0.080,0.091,0.107,0.132,0.158,0.183,0.216,0.266,0.323,0.384
4,NaN,NaN,NaN,NaN,0.022,0.024,0.026,0.030,0.033,0.039,...,0.056,0.063,0.074,0.092,0.110,0.127,0.151,0.185,0.225,0.267
5,NaN,NaN,NaN,NaN,NaN,0.016,0.018,0.020,0.022,0.026,...,0.038,0.043,0.051,0.063,0.075,0.087,0.103,0.127,0.154,0.183
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.014,0.015,0.018,...,0.026,0.029,0.034,0.042,0.051,0.059,0.070,0.086,0.104,0.124
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.010,0.012,...,0.018,0.020,0.023,0.029,0.035,0.040,0.047,0.058,0.071,0.084
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007,0.008,...,0.012,0.014,0.016,0.020,0.024,0.027,0.033,0.040,0.049,0.058


In [11]:
# Annually
rate_tree_annual = np.exp(rate_tree) - 1
rate_tree_annual.round(3)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,0.053,0.056,0.059,0.061,0.061,0.066,0.073,0.084,0.092,0.110,...,0.162,0.186,0.222,0.281,0.347,0.410,0.501,0.649,0.837,1.057
1,NaN,0.047,0.049,0.051,0.051,0.056,0.061,0.070,0.077,0.092,...,0.136,0.156,0.185,0.233,0.287,0.337,0.411,0.527,0.673,0.842
2,NaN,NaN,0.040,0.041,0.041,0.045,0.049,0.057,0.062,0.074,...,0.109,0.124,0.147,0.185,0.226,0.266,0.321,0.409,0.518,0.640
3,NaN,NaN,NaN,0.032,0.032,0.035,0.038,0.044,0.048,0.057,...,0.083,0.095,0.112,0.141,0.172,0.200,0.241,0.305,0.382,0.468
4,NaN,NaN,NaN,NaN,0.022,0.024,0.026,0.030,0.033,0.039,...,0.057,0.065,0.077,0.096,0.117,0.136,0.163,0.204,0.253,0.307
5,NaN,NaN,NaN,NaN,NaN,0.016,0.018,0.021,0.022,0.027,...,0.039,0.044,0.052,0.065,0.078,0.091,0.108,0.135,0.166,0.200
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.014,0.015,0.018,...,0.026,0.030,0.035,0.043,0.052,0.061,0.072,0.090,0.110,0.132
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.010,0.012,...,0.018,0.020,0.024,0.029,0.035,0.041,0.049,0.060,0.073,0.088
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007,0.008,...,0.012,0.014,0.016,0.020,0.024,0.028,0.033,0.041,0.050,0.059


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [12]:
FACE = 100
T = 5
compound = 4
cpn = forward_swap_rate
cpn_freq = 2

In [13]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + cpn / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, cpn)
# cash_flow_tree
# rate_tree.loc[cash_flow_tree.index, cash_flow_tree.columns]
bond_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.loc[cash_flow_tree.index, cash_flow_tree.columns], cftree=cash_flow_tree)
# bond_tree.round(2)
cash_flow_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
1,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
2,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
3,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
4,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
5,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
6,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
7,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0
8,0.0,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0,1.836106,0.0


In [14]:
bond_tree[0][0].round(2)

98.7

## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [15]:
swap_tree = 100 - bond_tree
swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,1.298017,2.510201,5.651032,7.100999,10.514763,12.302403,15.998539,17.910126,21.588536,23.430670,26.852265,28.119317,30.714301,31.064925,32.541225,31.157977,30.270626,26.228940,21.894036,12.521886
1,NaN,-2.490852,0.355579,1.495591,4.593869,6.061836,9.459525,11.105595,14.574140,16.266768,19.630740,20.939393,23.694948,24.312833,26.182436,25.317292,25.086261,21.815863,18.463795,10.464324
2,NaN,NaN,-4.054385,-3.202016,-0.411521,0.721689,3.781849,5.095263,8.259762,9.675920,12.830607,14.008841,16.745375,17.451150,19.557953,19.107870,19.507627,17.060254,14.792729,8.248142
3,NaN,NaN,NaN,-6.949468,-4.423434,-3.592023,-0.850388,0.131774,2.969946,4.059876,6.928394,7.872315,10.455301,11.084371,13.246466,13.034271,13.909433,12.191186,11.028090,6.074383
4,NaN,NaN,NaN,NaN,-7.491460,-6.903965,-4.426549,-3.729171,-1.179424,-0.399229,2.179681,2.862305,5.242770,5.707034,7.793617,7.654452,8.810023,7.583572,7.275270,3.743180
5,NaN,NaN,NaN,NaN,NaN,-9.260424,-6.976793,-6.490424,-4.156797,-3.612797,-1.258875,-0.783720,1.430159,1.751623,3.759422,3.655649,5.007195,4.140491,4.471613,2.009703
6,NaN,NaN,NaN,NaN,NaN,NaN,-8.756178,-8.418601,-6.237811,-5.862027,-3.669406,-3.344127,-1.252011,-1.037154,0.908178,0.824187,2.311537,1.698811,2.485335,0.786582
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.751617,-7.675925,-7.415643,-5.333510,-5.110507,-3.100953,-2.958217,-1.054689,-1.123230,0.459958,0.024436,1.126376,-0.046985
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.669714,-8.488107,-6.480672,-6.326032,-4.370632,-4.274099,-2.395438,-2.449179,-0.796274,-1.107060,0.212160,-0.604586


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [16]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=swap_tree.iloc[:tsteps,:tsteps])
swaption_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.711,2.914,4.787,7.444,10.515
1,NaN,0.554,1.121,2.268,4.594
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

In section 1, our price was roughly 1.69. Here, our price is about 1.71; they are close values.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [17]:
bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=swap_tree.iloc[:tsteps,:tsteps], style='american').round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.922,3.340,5.651,7.444,10.515
1,NaN,0.554,1.121,2.268,4.594
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y $\rightarrow$ 2y $\rightarrow$ 2y.

In [26]:
FACE = 100
T = 5
compound = 4
cpn = forward_swap_rate
cpn_freq = 4

In [27]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + cpn / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, cpn, cpn_freq=cpn_freq)
cash_flow_tree.iloc[:, :12] = 0
delayed_swap_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)], cftree=cash_flow_tree)
delayed_swap_tree.iloc[:, 12:] = 100 - delayed_swap_tree.iloc[:, 12:]
delayed_swap_tree.iloc[:, :12] = 0
delayed_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.546699,31.791240,32.372377,31.880900,30.119641,26.969433,21.787994,13.310502
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.558940,25.072782,26.042388,26.071234,24.958627,22.582447,18.373034,11.271489
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.640276,18.245265,19.447635,19.894874,19.405095,17.855262,14.718285,9.075285
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.378022,11.909910,13.164414,13.853174,13.832280,13.014783,10.971198,6.921123
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.187963,6.559486,7.735214,8.502063,8.754990,8.434566,7.234710,4.610935
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.391815,2.623912,3.718567,4.524629,4.968743,5.011977,4.443365,2.893086
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.278737,-0.150858,0.879769,1.708313,2.284896,2.584839,2.465871,1.680991
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.119637,-2.062255,-1.074490,-0.228675,0.441471,0.920442,1.112958,0.854939
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.383763,-3.371510,-2.409324,-1.547518,-0.809194,-0.204308,0.202838,0.302365


In [20]:
payoff = lambda p: p

delayed_swap_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:13,:13], undertree=delayed_swap_tree.iloc[:13,:13])
delayed_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [21]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

midcurve_swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=delayed_swap_tree.iloc[:tsteps,:tsteps])
midcurve_swaption_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.0,0.0,0.0,0.0,0.0
1,NaN,0.0,0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0
3,NaN,NaN,NaN,0.0,0.0
4,NaN,NaN,NaN,NaN,0.0


## 3.2

Price the **american** midcurve swaption 1y $\rightarrow$ 2y $\rightarrow$ 2y.

***

In [22]:
bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=delayed_swap_tree.iloc[:tsteps,:tsteps], style='american').round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.0,0.0,0.0,0.0,0.0
1,NaN,0.0,0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0
3,NaN,NaN,NaN,0.0,0.0
4,NaN,NaN,NaN,NaN,0.0
